# Working with Datastores in Azure Machine Learning SDK (v2)
This notebook contains examples for creating datastore using Azure Machine Learning SDK (v2). The samples are provided for the following storage types:

* Azure Blob Storage container
* Azure File share
* Azure Data Lake Storage Gen1
* Azure Data Lake Storage Gen2

The credentials property in these samples are redacted. Please replace the redacted account_key, sas_token, tenant_id, client_id and client_secret appropriately.

In [ ]:
#import required libraries
from azure.ml import MLClient
from azure.identity import InteractiveBrowserCredential
from azure.ml.entities import AzureBlobDatastore, AzureFileDatastore, AzureDataLakeGen1Datastore, AzureDataLakeGen2Datastore
from azure.ml.entities import CommandJob, JobInput, Environment

In [ ]:
#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(InteractiveBrowserCredential(), subscription_id, resource_group, workspace)

## Datastore for Azure Blob Storage container

In [ ]:
blob_datastore1 = AzureBlobDatastore(
    name='blob-example',
    description = 'Datastore pointing to a blob container.',
    account_name = 'mytestblobstore',
    container_name = 'data-container',
    credentials = {'account_key': 'XXXxxxXXXxXXXXxxXXXXXxXXXXXxXxxXxXXXxXXXxXXxxxXXxxXXXxXxXXXxxXxxXXXXxxxxxXXxxxxxxXXXxXXX'})
ml_client.create_or_update(blob_datastore1)

#create a SAS based blob datastore
blob_sas_datastore = AzureBlobDatastore(
    name='blob-sas-example',
    description = 'Datastore pointing to a blob container using SAS token.',
    account_name = 'mytestblobstore',
    container_name = 'data-container',
    credentials = {'sas_token': '?xx=XXXX-XX-XX&xx=xxxx&xxx=xxx&xx=xxxxxxxxxxx&xx=XXXX-XX-XXXXX:XX:XXX&xx=XXXX-XX-XXXXX:XX:XXX&xxx=xxxxx&xxx=XXxXXXxxxxxXXXXXXXxXxxxXXXXXxxXXXXXxXXXXxXXXxXXxXX'})
ml_client.create_or_update(blob_sas_datastore)

#create a datastore pointing to a blob container using wasbs protocol
blob_wasb_datastore = AzureBlobDatastore(
    name='blob-protocol-example',
    description = 'Datastore pointing to a blob container using wasbs protocol.',
    account_name = 'mytestblobstore',
    container_name = 'data-container',
    protocol='wasbs',
    credentials = {'account_key': 'XXXxxxXXXxXXXXxxXXXXXxXXXXXxXxxXxXXXxXXXxXXxxxXXxxXXXxXxXXXxxXxxXXXXxxxxxXXxxxxxxXXXxXXX'})
ml_client.create_or_update(blob_wasb_datastore)

#create a credential-less datastore pointing to a blob container
blob_credless_datastore = AzureBlobDatastore(
    name='blob-credless-example',
    description = 'Credential-less datastore pointing to a blob container.',
    account_name = 'mytestblobstore',
    container_name = 'data-container')
ml_client.create_or_update(blob_credless_datastore)


## Datastore for Azure File Share

In [ ]:
#Datastore pointing to an Azure File Share
file_datastore = AzureFileDatastore(
    name = 'file-example',
    description = 'Datastore pointing to an Azure File Share.',
    account_name = 'mytestfilestore',
    file_share_name = 'my-share',
    credentials = {'account_key': 'XXXxxxXXXxXXXXxxXXXXXxXXXXXxXxxXxXXXxXXXxXXxxxXXxxXXXxXxXXXxxXxxXXXXxxxxxXXxxxxxxXXXxXXX'})
ml_client.create_or_update(file_datastore)

#Datastore pointing to an Azure File Share using SAS token
file_sas_datastore = AzureFileDatastore(
    name = 'file-sas-example',
    description = 'Datastore pointing to an Azure File Share using SAS token.',
    account_name = 'mytestfilestore',
    file_share_name = 'my-share',
    credentials = {'sas_token': '?xx=XXXX-XX-XX&xx=xxxx&xxx=xxx&xx=xxxxxxxxxxx&xx=XXXX-XX-XXXXX:XX:XXX&xx=XXXX-XX-XXXXX:XX:XXX&xxx=xxxxx&xxx=XXxXXXxxxxxXXXXXXXxXxxxXXXXXxxXXXXXxXXXXxXXXxXXxXX'})
ml_client.create_or_update(file_sas_datastore)


# Datastore for Azure Data Lake Storage Gen1

In [ ]:
adlsg1_datastore = AzureDataLakeGen1Datastore(
    name = 'adls-gen1-example',
    description = 'Datastore pointing to an Azure Data Lake Storage Gen1.',
    store_name = 'mytestdatalakegen1', 
    credentials = {
        'tenant_id': 'XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX',
        'client_id': 'XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX',
        'client_secret': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'})
ml_client.create_or_update(adlsg1_datastore)

# Datastore for Azure Data Lake Storage Gen2

In [ ]:
adlsg2_datastore = AzureDataLakeGen2Datastore(
    name = 'adls-gen2-example',
    description = 'Datastore pointing to an Azure Data Lake Storage Gen2.',
    account_name = 'mytestdatalakegen2',
    filesystem = 'my-gen2-container',
    credentials = {
        'tenant_id': 'XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX',
        'client_id': 'XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX',
        'client_secret': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'})
ml_client.create_or_update(adlsg2_datastore)

# Using the datastore in a Job
Datastores can be used in a job like a `CommandJob` or a `PipelineJob`. In the below snippet, we will list the contents of a `datastore` in a `CommandJob`. We will use the default datastore `workspaceblobstore` which is created with any Azure Machine Learning Workspace.

The datastore can be used as a folder in the format `azureml://datastores/<datastore-name>/paths/<optional-path>`

In [ ]:
explore_datastore = CommandJob(
    command = 'ls ${{inputs.datastore}}',
    inputs = {'datastore': JobInput(folder='azureml://datastores/workspaceblobstore/paths/')},
    environment=Environment(image='python:latest'),
    compute = "cpu-cluster", 
    display_name="using-datastore"
)

#submit the command job
returned_job = ml_client.create_or_update(explore_datastore)
#get a URL for the status of the job
returned_job.services["Studio"].endpoint